In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from keras.models import Sequential, Model
from tensorflow.keras.layers import Input, GRU, Dense, Attention, Concatenate, Flatten, Dropout,Multiply, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.regularizers import l2
from datetime import timedelta
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.decomposition import PCA
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import VotingRegressor
from sklearn.datasets import make_regression
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit
import plotly.express as px
import tensorflow as tf
import math
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"D:\S2\data_tesis\23feb\BBCA.JK.csv")
df.index+=1
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Inflasi,Suku Bunga,Dividen,Hari
1,2015-01-02,2655,2655,2630,2645,2301.008301,29302500,6.96,7.75,0.0,Friday
2,2015-01-05,2630,2640,2625,2640,2296.658203,25434500,6.96,7.75,0.0,Monday
3,2015-01-06,2600,2640,2600,2620,2279.259521,48681500,6.96,7.75,0.0,Tuesday
4,2015-01-07,2610,2640,2610,2625,2283.608887,40334000,6.96,7.75,0.0,Wednesday
5,2015-01-08,2625,2630,2595,2595,2257.510742,54457000,6.96,7.75,0.0,Thursday


In [3]:
# Data jd ascending
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Inflasi,Suku Bunga,Dividen,Hari
0,2015-01-02,2655,2655,2630,2645,2301.008301,29302500,6.96,7.75,0.0,Friday
1,2015-01-05,2630,2640,2625,2640,2296.658203,25434500,6.96,7.75,0.0,Monday
2,2015-01-06,2600,2640,2600,2620,2279.259521,48681500,6.96,7.75,0.0,Tuesday
3,2015-01-07,2610,2640,2610,2625,2283.608887,40334000,6.96,7.75,0.0,Wednesday
4,2015-01-08,2625,2630,2595,2595,2257.510742,54457000,6.96,7.75,0.0,Thursday


In [4]:
# df.drop(['Inflasi', 'Suku Bunga', 'Dividen', 'Hari'], axis=1, inplace=True)

In [5]:
scaler_close = MinMaxScaler()
df['Close'] = scaler_close.fit_transform(df[['Close']])

In [6]:
df = pd.concat([df, pd.get_dummies(df['Hari'], prefix='Hari')], axis=1)

In [7]:
df.drop(['Hari'], axis=1, inplace=True)  # Drop the original 'Hari' column after encoding
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Inflasi,Suku Bunga,Dividen,Hari_Friday,Hari_Monday,Hari_Thursday,Hari_Tuesday,Hari_Wednesday
0,2015-01-02,2655,2655,2630,0.051061,2301.008301,29302500,6.96,7.75,0.0,True,False,False,False,False
1,2015-01-05,2630,2640,2625,0.050398,2296.658203,25434500,6.96,7.75,0.0,False,True,False,False,False
2,2015-01-06,2600,2640,2600,0.047745,2279.259521,48681500,6.96,7.75,0.0,False,False,False,True,False
3,2015-01-07,2610,2640,2610,0.048408,2283.608887,40334000,6.96,7.75,0.0,False,False,False,False,True
4,2015-01-08,2625,2630,2595,0.044430,2257.510742,54457000,6.96,7.75,0.0,False,False,True,False,False


In [8]:
features_to_normalize = df.columns.drop(['Date']).tolist()  # Adjust based on your dataset

In [9]:
features_to_normalize = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Inflasi', 'Suku Bunga', 'Dividen']
# features_to_normalize = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
scaler = MinMaxScaler()

In [10]:
df[features_to_normalize]= scaler.fit_transform(df[features_to_normalize])

In [11]:
# noise_intensity = 0.03  # Adjust the noise level
# df[features_to_normalize] += np.random.normal(0, noise_intensity, df[features_to_normalize].shape)

In [12]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Inflasi,Suku Bunga,Dividen,Hari_Friday,Hari_Monday,Hari_Thursday,Hari_Tuesday,Hari_Wednesday
0,2015-01-02,0.050667,0.040940,0.057333,0.051061,0.041161,0.027569,0.949495,1.0,0.0,True,False,False,False,False
1,2015-01-05,0.047333,0.038926,0.056667,0.050398,0.040604,0.023930,0.949495,1.0,0.0,False,True,False,False,False
2,2015-01-06,0.043333,0.038926,0.053333,0.047745,0.038380,0.045802,0.949495,1.0,0.0,False,False,False,True,False
3,2015-01-07,0.044667,0.038926,0.054667,0.048408,0.038936,0.037949,0.949495,1.0,0.0,False,False,False,False,True
4,2015-01-08,0.046667,0.037584,0.052667,0.044430,0.035599,0.051236,0.949495,1.0,0.0,False,False,True,False,False


In [13]:
# def create_sequences(df, sequence_length=60, target_column='Close'):
#     X, y = [], []
#     columns = df.columns.tolist()
#     target_column_index = columns.index(target_column)
#     df = df.values  # Convert to NumPy array for efficient processing
#     for i in range(len(df) - sequence_length):
#         X.append(df[i:i+sequence_length, :])
#         y.append(df[i+sequence_length, target_column_index])
#     return np.array(X), np.array(y)

In [14]:
def create_sequences_with_dates(features_to_normalize, dates, target, sequence_length=60):
    X, dates_seq, y = [], [], []
    for i in range(len(features_to_normalize) - sequence_length):
        X.append(features_to_normalize[i:(i + sequence_length)])
        dates_seq.append(dates[i + sequence_length - 1])  # Keep the last date of the sequence
        y.append(target[i + sequence_length])
    return np.array(X), np.array(dates_seq), np.array(y)

In [15]:
# processed_data = df.drop(columns=['Date'])

# X, y = create_sequences(processed_data)
# X = np.array(X, dtype=np.float32)
# y = np.array(y, dtype=np.float32)

In [16]:
# noise_intensity = 0.7  # Adjust the noise level
# df[features_to_normalize] += np.random.normal(0, noise_intensity, df[features_to_normalize].shape)

In [17]:
nan_columns = df.isna().any()

# Print columns that contain NaN values
print("Columns with NaN values:")
print(nan_columns[nan_columns == True].index.tolist())

# If you want to see the total count of NaN values in each column
nan_counts = df.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts[nan_counts > 0])

# To check for any NaN value in the entire DataFrame
if df.isna().any().any():
    print("\nThere are NaN values in the DataFrame.")
else:
    print("\nThere are no NaN values in the DataFrame.")

Columns with NaN values:
[]

Count of NaN values in each column:
Series([], dtype: int64)

There are no NaN values in the DataFrame.


In [18]:
df = df.fillna(0)
# df.dropna(how='all', inplace=True)
# df.fillna(method='ffill', inplace=True)

In [19]:
nan_columns = df.isna().any()

# Print columns that contain NaN values
print("Columns with NaN values:")
print(nan_columns[nan_columns == True].index.tolist())

# If you want to see the total count of NaN values in each column
nan_counts = df.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts[nan_counts > 0])

# To check for any NaN value in the entire DataFrame
if df.isna().any().any():
    print("\nThere are NaN values in the DataFrame.")
else:
    print("\nThere are no NaN values in the DataFrame.")

Columns with NaN values:
[]

Count of NaN values in each column:
Series([], dtype: int64)

There are no NaN values in the DataFrame.


In [20]:
kf = KFold(n_splits=5)
fold = 0
mae_list, mse_list, rmse_list, mape_list = [], [], [], []

for train_index, test_index in kf.split(X):
    fold += 1
    print(f'Fold #{fold}')
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

NameError: name 'X' is not defined

In [ ]:
model_gru = Sequential([
    GRU(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Dense(1)
])

# Compile the model
model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history_gru = model_gru.fit(X_train, y_train, epochs=80, batch_size=32 ,callbacks=[early_stopping], validation_split=0.2, verbose=1)

In [ ]:
train_gru_predictions = model_gru.predict(X_train)
test_gru_predictions = model_gru.predict(X_test)

# Combine GRU outputs with other features for Random Forest input
X_train_rf = np.hstack((X_train[:, -1, :], train_gru_predictions))  # Last time step features + GRU predictions
X_test_rf = np.hstack((X_test[:, -1, :], test_gru_predictions))

#

In [ ]:
# # # Define and train the Random Forest model
# random_forest_model = RandomForestRegressor(
#     n_estimators=40,
#     max_features=20,  # Mengurangi jumlah fitur maksimum yang diizinkan untuk split
#     min_samples_split=40,  # Meningkatkan jumlah minimum sampel untuk split
#     min_samples_leaf=40,  # Meningkatkan jumlah minimum sampel di setiap leaf
#     random_state=42
# )

# random_forest_model.fit(X_train_rf, y_train)

In [ ]:
random_forest_model= RandomForestRegressor(n_estimators=70, random_state=42)
random_forest_model.fit(X_train_rf, y_train)

In [ ]:
# predictions = model_rf.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape for RF

In [ ]:
non_zero_indices = y_test != 0

# Filter out zeros in both y_test and predictions
filtered_y_test = y_test[non_zero_indices]

In [ ]:
predictions = random_forest_model.predict(X_test_rf)

In [ ]:
filtered_predictions = predictions[non_zero_indices]
absolute_percentage_error = np.abs((filtered_y_test - filtered_predictions) / filtered_y_test)
mape = np.nanmean(absolute_percentage_error) * 100

In [ ]:
with np.errstate(divide='ignore', invalid='ignore'):
    mape_calculation = np.abs((y_test - predictions) / y_test)
    mape_calculation = mape_calculation[~np.isinf(mape_calculation)]
    mape = np.mean(mape_calculation) * 100

In [ ]:

# Calculate MAE
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error (MAE): {mae}")
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error (MSE): {mse}")
# Calculate RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate MAPE - Ensure y_test does not contain zeros to avoid division by zero errors
# mape = np.mean(np.abs((y_test - predictions_rf) / y_test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

# Calculate R-squared
r_squared = r2_score(y_test, predictions)
print(f"R-squared: {r_squared}")

In [ ]:
epsilon = 1e-2  # Nilai kecil untuk menghindari pembagian dengan nol
mape = np.mean(np.abs((y_test - predictions) / (y_test + epsilon))) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

In [ ]:
predicted_prices = model_gru.predict(X_test)  # Ini adalah output dari model GRU
predicted_prices_reshaped = predicted_prices.reshape(-1, 1)  # Pastikan bentuknya benar untuk inverse_transform

In [ ]:
# Inverse transform to original scale
next_day_prediction_original_scale = scaler_close.inverse_transform(predicted_prices_reshaped.reshape(-1, 1))

print(f"Next day's predicted price ( Original Scale): {next_day_prediction_original_scale[0][0]}")

In [ ]:
actual_prices_original = scaler_close.inverse_transform(y_test.reshape(-1, 1)).flatten()
predicted_prices_original = scaler_close.inverse_transform(predicted_prices_reshaped.reshape(-1, 1)).flatten()

In [ ]:
# plt.figure(figsize=(10, 5))
# save_path = 'D:\\S2\\Gambar 2\\gru-rf\\BBCA.jpg'
# plt.plot(actual_prices_original, label='Harga Asli', color='blue', marker='o')
# plt.plot(predicted_prices_original.flatten(), label='Harga Prediksi', color='red', linestyle='--', marker='x')
# plt.title('Harga Asli vs Harga Prediksi')
# plt.xlabel('Time Step')
# plt.ylabel('Harga Saham')
# plt.legend()

# plt.savefig(save_path)
# plt.grid(True)
# plt.show()